In [166]:
# feature extractoring and preprocessing data
import librosa
import numpy as np
import os
import pickle 
from keras import models

# Получаем предобработчик модели
with open('scaler.pickle', 'rb') as f:
    scaler = pickle.load(f)

In [167]:
# Загрузка модели
model = models.load_model('CNN.h5')

In [178]:
# Извлечение фич 
def GetFeatures(songname):
    y, sr = librosa.load(songname, mono=True, duration=30)
    chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
    spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
    spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
    zcr = librosa.feature.zero_crossing_rate(y)
    to_append = f'{np.mean(chroma_stft)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'   
    
    SoundArr = np.array(to_append.split())
    return SoundArr

In [207]:
# Получаем предсказание
def GetAnswer(SoundFeatures):
    # Предобрабатываем данные
    SoundArr_transformed = scaler.transform(SoundFeatures.reshape(1, -1))
    # Получаем прогноз
    predict = model.predict(SoundArr_transformed)
    if np.argmax(predict[0]) == 1:
        print("дрон, с вероятностью:",predict[0][1]*100, "%")
    elif np.argmax(predict[0]) == 0:
        print("не дрон, с вероятностью:",predict[0][0]*100, "%")

In [208]:
# Получаем предсказание для каждой записи
def GetAnswers(directory):
    for filename in os.listdir(f'{directory}'):
        songname = f'{directory}/{filename}'
        Features = GetFeatures(songname)
        print(f'На записи {songname}', end = " ")
        GetAnswer(Features)

In [209]:
GetAnswers('Test')

На записи Test/1_oTTcBwAB.wav дрон, с вероятностью: 99.25280809402466 %
На записи Test/5-243459-B-141.wav не дрон, с вероятностью: 99.99997615814209 %
На записи Test/5-243459-B-142.wav не дрон, с вероятностью: 100.0 %
На записи Test/5-243459-B-143.wav не дрон, с вероятностью: 99.99998807907104 %
На записи Test/5-243459-B-144.wav не дрон, с вероятностью: 100.0 %
На записи Test/5-243635-A-290.wav не дрон, с вероятностью: 99.10539984703064 %
На записи Test/5-243635-A-291.wav не дрон, с вероятностью: 99.89602565765381 %
